In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("filled.csv")
df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,0.0,5.0,-1.00,0.0,2.0,8.0,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,0.0,4.0,-1.00,0.0,3.0,5.0,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,Student,5.0,0.0,8.97,2.0,0.0,5.5,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,0.0,5.0,-1.00,0.0,1.0,5.0,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,0.0,1.0,-1.00,0.0,1.0,5.5,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [4]:
df.columns

Index(['id', 'Name', 'Gender', 'Age', 'City',
       'Working Professional or Student', 'Profession', 'Academic Pressure',
       'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction',
       'Sleep Duration', 'Dietary Habits', 'Degree',
       'Have you ever had suicidal thoughts ?', 'Work/Study Hours',
       'Financial Stress', 'Family History of Mental Illness', 'Depression'],
      dtype='object')

In [5]:
X = df.drop(columns=['Depression','id', 'Name'])
y = df['Depression']

X.columns

Index(['Gender', 'Age', 'City', 'Working Professional or Student',
       'Profession', 'Academic Pressure', 'Work Pressure', 'CGPA',
       'Study Satisfaction', 'Job Satisfaction', 'Sleep Duration',
       'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?',
       'Work/Study Hours', 'Financial Stress',
       'Family History of Mental Illness'],
      dtype='object')

In [6]:
y

0         0
1         1
2         1
3         1
4         0
         ..
140695    1
140696    0
140697    0
140698    0
140699    1
Name: Depression, Length: 140700, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112560, 17), (28140, 17), (112560,), (28140,))

In [8]:
train_number_columns = X_train.select_dtypes(include=['number']).columns
print(train_number_columns)
print('-----------------------------------------------')
train_str_columns = X_train.select_dtypes(include=['object']).columns
print(train_str_columns)

Index(['Age', 'Academic Pressure', 'Work Pressure', 'CGPA',
       'Study Satisfaction', 'Job Satisfaction', 'Sleep Duration',
       'Work/Study Hours', 'Financial Stress'],
      dtype='object')
-----------------------------------------------
Index(['Gender', 'City', 'Working Professional or Student', 'Profession',
       'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?',
       'Family History of Mental Illness'],
      dtype='object')


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
number_pipline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
str_pipline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])
preprocessing = ColumnTransformer([
    ('number', number_pipline, train_number_columns),
    ('str', str_pipline, train_str_columns)
])
# fit
# -> transfrom
# = fit_transform
x_train_processed = preprocessing.fit_transform(X_train)
x_test_processed = preprocessing.transform(X_test)


In [10]:
x_train_processed.shape, x_test_processed.shape

((112560, 17), (28140, 17))

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import joblib

cls = SVC(C=300)
cls.fit(x_train_processed, y_train)




SVC(C=300)

In [12]:
from sklearn.metrics import classification_report
y_predict = cls.predict(x_test_processed)
print(classification_report(y_test, y_predict))
print(classification_report(y_train,cls.predict(x_train_processed) ))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     22986
           1       0.84      0.81      0.83      5154

    accuracy                           0.94     28140
   macro avg       0.90      0.89      0.89     28140
weighted avg       0.94      0.94      0.94     28140

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     92147
           1       0.85      0.81      0.83     20413

    accuracy                           0.94    112560
   macro avg       0.90      0.89      0.90    112560
weighted avg       0.94      0.94      0.94    112560



In [13]:
cls1 = SVC(C=10)
cls1.fit(x_train_processed, y_train)

SVC(C=10)

In [14]:
y_predict = cls1.predict(x_test_processed)
print(classification_report(y_test, y_predict))
print(classification_report(y_train,cls1.predict(x_train_processed) ))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     22986
           1       0.84      0.80      0.82      5154

    accuracy                           0.94     28140
   macro avg       0.90      0.89      0.89     28140
weighted avg       0.94      0.94      0.94     28140

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     92147
           1       0.85      0.80      0.82     20413

    accuracy                           0.94    112560
   macro avg       0.90      0.88      0.89    112560
weighted avg       0.94      0.94      0.94    112560



In [15]:
cls2 = SVC(C=0.1)
cls2.fit(x_train_processed, y_train)

SVC(C=0.1)

In [16]:
y_predict = cls2.predict(x_test_processed)
print(classification_report(y_test, y_predict))
print(classification_report(y_train,cls2.predict(x_train_processed) ))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93     22986
           1       0.75      0.59      0.66      5154

    accuracy                           0.89     28140
   macro avg       0.83      0.77      0.79     28140
weighted avg       0.88      0.89      0.88     28140

              precision    recall  f1-score   support

           0       0.91      0.95      0.93     92147
           1       0.74      0.59      0.66     20413

    accuracy                           0.89    112560
   macro avg       0.83      0.77      0.80    112560
weighted avg       0.88      0.89      0.88    112560



In [17]:
cls3 = SVC(C=1)
cls3.fit(x_train_processed, y_train)

SVC(C=1)

In [18]:
y_predict = cls3.predict(x_test_processed)
print(classification_report(y_test, y_predict))
print(classification_report(y_train,cls3.predict(x_train_processed) ))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     22986
           1       0.85      0.76      0.80      5154

    accuracy                           0.93     28140
   macro avg       0.90      0.86      0.88     28140
weighted avg       0.93      0.93      0.93     28140

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     92147
           1       0.85      0.76      0.80     20413

    accuracy                           0.93    112560
   macro avg       0.90      0.86      0.88    112560
weighted avg       0.93      0.93      0.93    112560



In [20]:
cls4 = SVC(kernel="linear")
cls4.fit(x_train_processed, y_train)

SVC(kernel='linear')

In [21]:
y_predict = cls4.predict(x_test_processed)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     22986
           1       0.84      0.81      0.82      5154

    accuracy                           0.94     28140
   macro avg       0.90      0.89      0.89     28140
weighted avg       0.94      0.94      0.94     28140



In [ ]:
cls4 = SVC(C=10,kernel="linear")
cls4.fit(x_train_processed, y_train)

In [19]:

overlap = np.intersect1d(x_train_processed, x_test_processed)
print(f"Số mẫu trùng lặp: {len(overlap)}")


Số mẫu trùng lặp: 490
